In [2]:
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError
import hashlib
from dotenv import load_dotenv  # {{ edit_1 }}
import os, csv
load_dotenv()


True

In [17]:
client = MailchimpMarketing.Client()

list_id = "b578d43584"

# Set your API key and server prefix
client.set_config({
    "api_key": os.getenv("MAILCHIMP_API_KEY"),  # {{ edit_2 }}
    "server": "us12"  # e.g., "us6"
})

In [40]:
# Initialize variables for pagination
count = 100  # Max allowed per request is 1000, but you can adjust as needed
offset = 0
all_members = []

try:
    while True:
        # Fetch list members with pagination
        response = client.lists.get_list_members_info(list_id, count=count, offset=offset)
        members = response['members']
        all_members.extend(members)
        
        # Check if we have retrieved all members
        total_items = response['total_items']
        if len(all_members) >= total_items:
            break
        
        # Move to the next page
        offset += count

except ApiClientError as error:
    print("Error: {}".format(error.text))

print(f"Total members retrieved: {len(all_members)}")

Total members retrieved: 7940


In [19]:
# Function to generate the subscriber hash from email
def get_subscriber_hash(email):
    return hashlib.md5(email.lower().encode('utf-8')).hexdigest()

In [50]:
non_openers = []
processed_members = 0

# Verificar a atividade de abertura de email para cada membro
for member in all_members:
    processed_members += 1
    email = member['email_address']
    subscriber_hash = get_subscriber_hash(email)

    try:
        # Obter atividade do membro
        activity = client.lists.get_list_member_activity(list_id, subscriber_hash)
        has_opened = any(event['action'] == 'open' for event in activity['activity'])
        
        # Se nunca abriu um email, adicionar à lista de não-abertos
        if not has_opened:
            non_openers.append(email)

    except ApiClientError as error:
        print(f"Erro ao buscar atividade para {email}: {error.text}")
    print(f"Members processed: {processed_members}, Non-openers: {len(non_openers)}", end='\r')


Erro ao buscar atividade para danila.badat@gmail.com: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Erro ao buscar atividade para sulhard@hotmail.com: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [71]:
# Get all fields from the first member
fields = all_members[0].keys()

# Write the data to a CSV file
with open('non_openers_data.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()
    for member in all_members:
        if member['email_address'] in non_openers:
            writer.writerow(member)


In [11]:
import pandas as pd
all_members = pd.read_csv("all_members.csv")
non_openers = pd.read_csv("non_openers_data.csv")["email_address"].to_list()

In [35]:
# Add a tag to all members who are in the non_openers list
failed_contact_ids = []
processed_members_count = 0
total_members = len(all_members)
for index, member in all_members.iterrows():
    processed_members_count += 1
    if member['email_address'] in non_openers:
        # print(member['contact_id'])
        response = client.lists.update_list_member_tags(list_id, get_subscriber_hash(member['email_address']), {"tags": [{"name": "Non-Opener", "status": "active"}]})
        # Check if the response indicates success
        if response.status_code == 204:
            pass
        else:
            print("Failed to add tag:", response.content)  # Print error if tag addition fails
            failed_contact_ids.append(member['contact_id'])
        print(f"{processed_members_count} of {total_members} members processed", end='\r')